In [1]:
from matplotlib import pyplot as plt
from modnet.hyper_opt.fit_genetic import FitGenetic
from modnet.models import EnsembleMODNetModel
from modnet.preprocessing import MODData
from pathlib import Path
from plotly.subplots import make_subplots
from scipy.stats import gaussian_kde, spearmanr
from sklearn.metrics import mean_absolute_error, root_mean_squared_error, mean_absolute_percentage_error
from sklearn.model_selection import KFold

import copy
import json
import numpy as np
import os
import pandas as pd
import plotly.graph_objs as go

2025-01-05 15:44:28.322828: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-01-05 15:44:28.432587: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-01-05 15:44:28.435488: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/vtrinquet/local/lib:
2025-01-05 15:44:28.435505: I tensorflow/compiler/xla/stream_execut

In [2]:
cur_v = int(Path(os.getcwd()).parent.name.split("_")[0])
print(cur_v)

19


# Training the model and loading it

In [3]:
targets = ["dKP"]
experiment_name = "GA_Rf0_Nstd5-" + "-".join(targets)

model_path = (
    Path("..") / "models" / "production" / f"{experiment_name}_prod_v{cur_v}.pkl.gz"
)

if model_path.exists():
    os.system(f"gunzip {model_path}")
    model_path = str(model_path).replace(".gz", "")
    model = EnsembleMODNetModel.load(model_path)
    os.system(f"gzip {model_path}")
else:
    # Load the MODData
    path_md = f'../data/mod.data_dKP_fastfeat_n_Eg_pgnn_featselec_v{cur_v}'
    os.system(f"gunzip {path_md}.gz")
    md = MODData.load(path_md)
    os.system(f"gzip {path_md}")
    print(f"{md.df_featurized.shape = }")

    ga = FitGenetic(md, sample_threshold=len(md.df_featurized)+1)
    model = ga.run(
        size_pop=20, # dflt
        num_generations=10, # dflt
        nested=5, # dflt = 5
        n_jobs=2,
        early_stopping=4, # dflt
        refit=0, # dflt = 5
        fast=False,
    )
    model.save(model_path)

2025-01-05 15:44:31.533567: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/vtrinquet/local/lib:
2025-01-05 15:44:31.533589: W tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:265] failed call to cuInit: UNKNOWN ERROR (303)
2025-01-05 15:44:31.533603: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (PC-SE22-351): /proc/driver/nvidia/version does not exist
2025-01-05 15:44:31.533772: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


2025-01-05 15:44:33,098 - modnet - INFO - Loaded <modnet.models.ensemble.EnsembleMODNetModel object at 0x7da36195e200> object, created with modnet version 0.4.4


# Predicting the full search space

In [4]:
target_name = model.targets[0][0][0]
print(f"{target_name}")

dKP


In [5]:
path_df_full_pred = "../data/df_full_pred.pkl.gz"
if Path(path_df_full_pred).exists():
    df_full_pred = pd.read_pickle(path_df_full_pred)
else:

    # Load the fast matminer features of the full search space
    path_df_full_feat = "../../../00_prepa_dbs/full_search_space/data/df_full_feat.pkl.gz"
    df_full_feat = pd.read_pickle(path_df_full_feat)
    print(f"{df_full_feat.shape = }")

    # Load the pGNN features of the full search space
    path_df_pgnn_mmv1 = "../../../00_prepa_dbs/full_search_space/data/df_mmv1.pkl.gz"
    path_df_pgnn_ofm = "../../../00_prepa_dbs/full_search_space/data/df_ofm.pkl.gz"
    path_df_pgnn_mvl32 = "../../../00_prepa_dbs/full_search_space/data/df_mvl32.pkl.gz"
    df_pgnn = pd.concat([pd.read_pickle(path) for path in [path_df_pgnn_mmv1, path_df_pgnn_ofm, path_df_pgnn_mvl32]], axis=1).drop(['structure'], axis=1, errors='ignore')
    print(f"{df_pgnn.shape = }")
    assert len(df_full_feat)==len(df_pgnn)

    df_full_feat = pd.concat([df_full_feat, df_pgnn], axis=1)
    print(f"{df_full_feat.shape = }")

    # Drop the entries present in training
    path_md_training = f'../data/mod.data_dKP_fastfeat_n_Eg_pgnn_featselec_v{cur_v}'
    os.system(f"gunzip {path_md_training}.gz")
    df_full_feat = df_full_feat.drop(MODData.load(path_md_training).df_featurized.index, axis=0, errors='ignore')
    os.system(f"gzip {path_md_training}")
    print(f"{df_full_feat.shape = }")

    # Create a MODData for the full search space
    md = MODData(
        df_featurized=df_full_feat,
    )

    # Predict the full search space
    df_full_pred, df_unc = model.predict(md, return_unc=True)
    df_full_pred[f"{target_name}_unc"] = df_unc[target_name]

    # calibrate the uncertainties
    experiment_name_bmk = "GA_Rf0_Nstd5_5fold-" + "-".join(targets)
    path_bmk_predictions = (Path("..") / "figures/bmk" / f"bmk_{experiment_name_bmk}_{target_name}.pkl.gz")
    df_bmk_pred = pd.read_pickle(path_bmk_predictions)
    scaling_ma = df_bmk_pred[f"{target_name}_unc_cal_ma"]/df_bmk_pred[f"{target_name}_unc"]
    scaling_ma = scaling_ma.value_counts(ascending=False).keys()[0]
    scaling_nll = df_bmk_pred[f"{target_name}_unc_cal_nll"]/df_bmk_pred[f"{target_name}_unc"]
    scaling_nll = scaling_nll.value_counts(ascending=False).keys()[0]

    df_full_pred[f"{target_name}_unc_cal_ma"] = df_full_pred[f"{target_name}_unc"]*scaling_ma
    df_full_pred[f"{target_name}_unc_cal_nll"] = df_full_pred[f"{target_name}_unc"]*scaling_nll

    df_full_pred.to_pickle(path_df_full_pred)

print(f"{df_full_pred.shape = }")
display(df_full_pred.head())

df_full_feat.shape = (33443, 216)
df_pgnn.shape = (33443, 1074)
df_full_feat.shape = (33443, 1290)
2025-01-05 15:44:56,835 - modnet - INFO - Loaded <modnet.preprocessing.MODData object at 0x7da360260e50> object, created with modnet version 0.4.4
df_full_feat.shape = (31124, 1290)
973/973 [==============================] - 1s 922us/step
df_full_pred.shape = (31124, 4)


,dKP,dKP_unc,dKP_unc_cal_ma,dKP_unc_cal_nll
mp-759792,1.331904,0.386815,0.563414,8.338441
mp-660503,5.468415,3.766065,5.485450,81.183861
mp-34982,32.413658,15.832521,23.060810,341.296600
mp-1229155,7.266753,3.507335,5.108597,75.606491
mp-1229187,103.444359,21.803894,31.758394,470.019592


# Visualizing the predictions

In [6]:
# Load df_outputs_filtout
path_df_outputs_filtout = Path(f'../data/df_outputs_filtout.pkl.gz')
df_outputs_filtout = pd.read_pickle(path_df_outputs_filtout)
print(f"{df_outputs_filtout.shape = }")

df_outputs_filtout.shape = (2518, 26)


In [7]:
# Separate the different DBs of the full search space
def to_which_db(index):
    if "agm" in index:
        return "Alexandria"
    elif "mp" in index:
        return "Materials Project"
    elif "mvc" in index: # https://docs.materialsproject.org/frequently-asked-questions#in-the-past-i-have-seen-material_ids-that-start-with-mvc-what-are-these
        return "Materials Project"
    else:
        raise Exception(f"The index {index} does not correspond to any DB.")

df_pred_agm = df_full_pred.filter(df_full_pred.groupby(by=to_which_db).groups['Alexandria'], axis=0)
df_pred_mp = df_full_pred.filter(df_full_pred.groupby(by=to_which_db).groups['Materials Project'], axis=0)

# Load the std data of the full search space
path_df_full_std = "../../../00_prepa_dbs/full_search_space/data/df_full_std.pkl.gz"
df_full_std = pd.read_pickle(path_df_full_std)

df_std_agm = df_full_std.filter(df_full_std.groupby(by=to_which_db).groups['Alexandria'], axis=0).filter(df_pred_agm.index, axis=0)
df_std_mp = df_full_std.filter(df_full_std.groupby(by=to_which_db).groups['Materials Project'], axis=0).filter(df_pred_mp.index, axis=0)

print(f"{df_full_pred.shape = }")
assert len(df_full_pred) == np.sum(len(df_pred_agm) + len(df_pred_mp))
print(f"{df_pred_agm.shape = }")
print(f"{df_std_agm.shape = }")
print(f"{df_pred_mp.shape = }")
print(f"{df_std_mp.shape = }")


df_full_pred.shape = (31124, 4)
df_pred_agm.shape = (21266, 4)
df_std_agm.shape = (21266, 6)
df_pred_mp.shape = (9858, 4)
df_std_mp.shape = (9858, 6)


In [8]:
# dKP-Eg-KDE with log-lin inset

# ==============================================================================================================================
fig = make_subplots(rows=1, cols=2, shared_yaxes=True, horizontal_spacing=0.02, column_widths=[0.8,0.2])

# ROW 1 COL 1 =================================================================================================================
fig.add_trace(go.Scatter(x=df_outputs_filtout[df_outputs_filtout['origin']=='Naccarato']['src_bandgap'],
                         y=df_outputs_filtout[df_outputs_filtout['origin']=='Naccarato']['dKP'],
                         mode='markers',
                         marker = dict(color = 'royalblue',
                                       size=7,
                                       #df_outputs_filtout['n'],
                                    #    showscale=True,
                                    #    colorbar = dict(title = '<i>n<sub>s</sub></i>', 
                                    #                    title_font_size = 36,
                                    #                    orientation = 'v',
                                    #                 #    tickvals=[1,2,3,4,5,6],
                                    #                    x = 0.9,
                                    #                    y=0.6, 
                                    #                    len = 0.9),
                                    #    colorscale = 'plasma', 
                                       symbol='x',
                                       ),
                         showlegend=True,
                         name=f'Nacc',
                         text=df_outputs_filtout[df_outputs_filtout['origin']=='Naccarato'].index.tolist(),
                         opacity=0.5
                         ))

fig.add_trace(go.Scatter(x=df_outputs_filtout[df_outputs_filtout['origin']!='Naccarato']['src_bandgap'],
                         y=df_outputs_filtout[df_outputs_filtout['origin']!='Naccarato']['dKP'],
                         mode='markers',
                         marker = dict(symbol='x',
                                       size=7,
                                       color = 'red',
                                       showscale=False,
                                       line=dict(width=2, color="black")
                                       ),
                         showlegend=True,
                         name=f'v{cur_v-1} outputs',
                         text=[mpid for mpid in df_outputs_filtout[df_outputs_filtout['origin']!='Naccarato'].index.values],
                         opacity=0.5
                         ))

fig.add_trace(go.Scatter(x=df_std_mp['bandgap'],
                         y=df_pred_mp['dKP'],
                         error_y={'array': df_pred_mp['dKP_unc'].tolist(), 'visible': True},
                         mode='markers',
                         marker = dict(size=7,
                                       color = 'lightgreen',
                                       showscale=False,
                                       line=dict(width=2, color="black")
                                       ),
                         showlegend=True,
                         name=f'MP',
                         text=df_pred_mp.index.tolist(),
                         opacity=0.3
                         ))
fig.add_trace(go.Scatter(x=df_std_agm['bandgap'],
                         y=df_pred_agm['dKP'],
                         error_y={'array': df_pred_agm['dKP_unc'].tolist(), 'visible': True},
                         mode='markers',
                         marker = dict(size=7,
                                       color = 'orange',
                                       showscale=False,
                                       line=dict(width=2, color="black")
                                       ),
                         showlegend=True,
                         name=f'AGM',
                         text=df_pred_agm.index.tolist(),
                         opacity=0.3
                         ))


# AXES
fig.update_xaxes(title = '<i>E<sub>g</sub></i> (eV)',
                 title_font_size=36,
                 range = [-0.1, 10],
                 row=1, col=1)
fig.update_yaxes(title = '<i>d</i><sub>KP</sub> (pm/V)',
                 title_font_size=36)


fig.update_layout(font={'family':'Arial', 'size': 20},
                 )

# ROW 1 COL 2 =================================================================================================================
density = gaussian_kde(np.reshape(df_full_pred['dKP'].values, (1,len(df_full_pred))))
density.covariance_factor = lambda : .02 #Smoothing parameter
density._compute_covariance()

x_vals = np.linspace(min(df_full_pred['dKP'].values),
                     max(df_full_pred['dKP'].values),
                     200) # Specifying the limits of our data
kde_dist = density(x_vals)

fig.add_trace(go.Scatter(x=kde_dist, 
                         y=x_vals, 
                         mode='lines', 
                         marker_color='indianred',
                         fill='tozerox',
                         showlegend=False),
              row=1, col=2)

fig.update_xaxes(title = 'Distribution',
                 title_font_size=36,
                 row=1, col=2)
fig.update_yaxes(title = '', row=1, col=2)

# # INSET =======================================================================================================================
# inset = copy.deepcopy(fig.data[0])
# inset.xaxis = 'x3'
# inset.yaxis = 'y3'

# inset_candidates_v1 = copy.deepcopy(fig.data[1])
# inset_candidates_v1.xaxis = 'x3'
# inset_candidates_v1.yaxis = 'y3'
# inset

# fig.update_layout(
#     xaxis3=dict(domain      = [0.30, 0.75],
#                 anchor      = 'y3',
#                 range       = [-0.1, 8.4],
#                 linecolor   = 'black'
#                ),
#     yaxis3=dict(domain      = [0.60, 0.98],
#                 anchor      = 'x3',
#                 range       = [-3.5, 2.6],
#                 type        = 'log',
#                 tickvals    = [0.001, 0.01, 1, 100],
#                 linecolor   = 'black'
#     ))

# fig.add_trace(inset)
# fig.add_trace(inset_candidates_v1)

fig.update_xaxes(showspikes=True, spikecolor="gray", spikethickness=2, spikesnap="cursor", spikemode="across")
fig.update_yaxes(showspikes=True, spikecolor="gray", spikethickness=2, spikesnap="cursor", spikemode='across')
fig.update_layout(hoverdistance=5)

# THEME OF GRAPH
fig.update_layout(template='simple_white')

fig.update_layout(
    width=1200,
    # width=1000,
    height=500
)

fig.show()


# Write out HTML file 
# It is important to use this syntax with the above "html_str" to create the html file
# with open(f'{Path_fig_HTML}/dRMS-Eg_KDE_log-lin-inset.html', 'w') as f:
#     f.write(html_str)
name_fig = "dKP-Eg_KDE_full_pred"
if not Path(f'../figures/pdf/{name_fig}.pdf').exists():
    fig.write_image(f'../figures/pdf/{name_fig}.pdf')
    fig.write_image(f'../figures/svg/{name_fig}.svg')
    fig.write_image(f'../figures/png/{name_fig}.png', scale=10)
    fig.write_html(f'../figures/html/{name_fig}.html')
    os.system(f"gzip ../figures/html/{name_fig}.html")
